# 1_animal_game

In [7]:
import ipykernel, sys

print(ipykernel.__version__)
print(sys.version)


class AnimalNode:
    def __init__(self, question):
        self.question = question
        self.yes_child = None
        self.no_child = None

    # Return 'a' or 'an' as appropriate for this animal's name.
    def article(self):
        the_vowel = ["a", "e", "i", "o", "u"]
        return "an" if self.question.lower()[0] in the_vowel else "a"

    # Return the animal's name with an article.
    def full_name(self):
        return self.article() + " " + self.question

    # Return True if this is a leaf node (which represents an animal).
    def is_leaf(self):
        return self.no_child == None

    def want_to_play_again(self):
        user_input = input("Do you want to play again? y/n")

        return True if user_input == "y" else False

    def handle_adding_new_animal(self):  # == Ostrish
        user_input = input("What was your animal name?")  # Tarantula
        newAnimalNode = AnimalNode(user_input)
        user_input = input(
            "What question can I ask to differentiate between a "
            + self.full_name()
            + " and "
            + newAnimalNode.full_name()
            + "?"
        )
        newQuestionNode = AnimalNode(user_input)  # Feathered
        user_input = input(self.full_name() + " is " + newQuestionNode.question())
        if user_input == "y":
            newQuestionNode.yes_child = self
            newQuestionNode.no_child = newAnimalNode
        else:
            newQuestionNode.yes_child = newAnimalNode
            newQuestionNode.no_child = self

        global_root = newQuestionNode

        self.want_to_play_again()

    def handle_leaf(self):
        user_input = input("Is it " + self.full_name() + "?")

        if user_input == "y":
            print("Good Job!\n")
        if user_input == "n":
            print("You are wrong!\n")
            self.handle_adding_new_animal()

        return self.want_to_play_again()

    def handle_not_leaf(self):
        user_input = input("Does it has " + self.full_name() + "?")

        if user_input == "y":
            return self.yes_child()
        if user_input == "n":
            return self.no_child()

        return False

    # Ask this node's question and move to the appropriate child.
    # Return True if we keep playing.
    def ask_question(self):
        if self.is_leaf():
            return self.handle_leaf()
        else:
            return self.handle_not_leaf()


# Initialize the tree.
global_root = AnimalNode("Ostrich")


def play_game():
    # Display instructions.
    print("Answer y for Yes, n for No, and q for Quit\n")

    # Repeat until the user quits.
    while global_root.ask_question():
        pass


play_game()

6.27.1
3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
Answer y for Yes, n for No, and q for Quit

